In [ ]:
import json
import os

INPUT_FILE = "cleaned_large_data.json"

def load_data(filename):
    if not os.path.exists(filename):
        print(f"[ERROR] '{filename}' not found. Please run data_cleaning.ipynb first.")
        return {}
    with open(filename, "r") as f:
        return json.load(f)

data = load_data(INPUT_FILE)

if data:
    id_to_name = {u['id']: u['name'] for u in data['users']}
    page_lookup = {p['id']: p['name'] for p in data['pages']}
    print(f"✅ System Ready: Loaded {len(data['users'])} users and {len(data['pages'])} pages.")

✅ System Ready: Loaded 396 users and 50 pages.


In [ ]:
def find_people_you_may_know(user_id, data, limit=5):
    """
    Returns a list of (User_ID, Mutual_Count) tuples.
    Logic: Friends of Friends who are not already friends.
    """
    # 1. List (User ID -> Set of Friends)
    user_friends = {}
    for user in data['users']:
        user_friends[user['id']] = set(user['friends'])
    
    if user_id not in user_friends:
        return []

    target_friends = user_friends[user_id]
    suggestions = {}

    # 2. The "Friend of a Friend" Loop
    for friend_id in target_friends:
        # Get the friend's connections
        friends_of_friend = user_friends.get(friend_id, set())
        
        for candidate in friends_of_friend:
            # 1. Not myself
            # 2. Not already my friend
            if candidate != user_id and candidate not in target_friends:
                suggestions[candidate] = suggestions.get(candidate, 0) + 1

    # 3. Sort by Mutual Friends (Highest first)
    # Returns list of tuples: [(User_ID, Count), ...]
    sorted_suggestions = sorted(suggestions.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_suggestions[:limit]

In [ ]:
def find_pages_you_might_like(user_id, data, limit=5):
    """
    Returns a list of (Page_ID, Score) tuples.
    Logic: Users who liked similar pages (Weighted Voting).
    """
    # 1. Database (User ID -> Set of Likes)
    user_likes = {}
    for user in data['users']:
        user_likes[user['id']] = set(user['liked_pages'])
        
    if user_id not in user_likes:
        return []

    my_likes = user_likes[user_id]
    page_scores = {}

    # 2. Compare against every other user
    for other_user, other_likes in user_likes.items():
        if other_user == user_id:
            continue
            
        common_interests = my_likes.intersection(other_likes)
        similarity_score = len(common_interests)
        
        if similarity_score > 0:
            for page in other_likes:
                if page not in my_likes:
                    
                    page_scores[page] = page_scores.get(page, 0) + similarity_score

    # 3. Sort by Score (Highest first)
    sorted_pages = sorted(page_scores.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_pages[:limit]

In [ ]:
def generate_user_dashboard(user_id):
    """
    Runs both engines and prints a professional report.
    """
    # Check if user exists
    user_name = id_to_name.get(user_id)
    if not user_name:
        print(f"❌ Error: User ID {user_id} not found.")
        return

    print("="*60)
    print(f"🚀 RECOMMENDATION DASHBOARD FOR: {user_name.upper()} (ID: {user_id})")
    print("="*60)
    
    #1: PEOPLE
    friend_recs = find_people_you_may_know(user_id, data, limit=5)
    
    print(f"\n👥 PEOPLE YOU MAY KNOW (Top 5)")
    print(f"{'Name':<25} | {'Mutual Friends':<15}")
    print("-" * 45)
    
    if friend_recs:
        for fid, count in friend_recs:
            fname = id_to_name.get(fid, "Unknown")
            print(f"{fname:<25} | {count:<15}")
    else:
        print("No suggestions found (Try adding more friends first!)")

    #2: PAGES
    page_recs = find_pages_you_might_like(user_id, data, limit=5)
    
    print(f"\n📄 PAGES YOU MIGHT LIKE (Top 5)")
    print(f"{'Page Name':<35} | {'Relevance Score':<15}")
    print("-" * 55)
    
    if page_recs:
        for pid, score in page_recs:
            pname = page_lookup.get(pid, "Unknown Page")
            print(f"{pname:<35} | {score:<15}")
    else:
        print("No page suggestions (Try liking some pages first!)")
        
    print("\n" + "="*60)

In [ ]:

TARGET_USER_ID = 33

generate_user_dashboard(TARGET_USER_ID)

🚀 RECOMMENDATION DASHBOARD FOR: ANIKA ARJUN (ID: 333)

👥 PEOPLE YOU MAY KNOW (Top 5)
Name                      | Mutual Friends 
---------------------------------------------
Aditya Arjun              | 2              
Unknown                   | 2              
Unknown                   | 2              
Unknown                   | 2              
Aditya Anika              | 2              

📄 PAGES YOU MIGHT LIKE (Top 5)
Page Name                           | Relevance Score
-------------------------------------------------------
Data Science Experts                | 179            
Web3 Community                      | 178            
SQL Forum                           | 174            
Web3 Experts                        | 172            
Cyber Security Forum                | 167            

